<a href="https://colab.research.google.com/github/JacopoMangiavacchi/Swift-TensorFlow-Sample-Notebooks/blob/master/XOR_Swift_TensorFlow_Fast_2-10-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import TensorFlow

In [0]:
struct XOR: Layer {
    var l1, l2: Dense<Float>
    init() {
        l1 = Dense<Float>(
            inputSize: 2,
            outputSize: 10,
            activation: relu,
            seed: (0xfffffff, 0xfeeff)
        )
        l2 = Dense<Float>(
            inputSize: 10,
            outputSize: 1,
            activation: relu,
            seed: (0xfeffeffe, 0xfffe)
        )
    }
  
    @differentiable
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        let h1 = l1.applied(to: input, in: context)
        return l2.applied(to: h1, in: context)
    }
}

In [0]:
var model = XOR()
let optimizer = SGD(for: model, learningRate: 0.03, scalarType: Float.self)

let x: Tensor<Float> = [[0, 0], [0, 1], [1, 0], [1, 1]]
let y: Tensor<Float> = [[0], [1], [1], [0]]

let trainingContext = Context(learningPhase: .training)
for epoch in 1...1000 {
    let 𝛁model = model.gradient { m -> Tensor<Float> in
        let ŷ = m.applied(to: x, in: trainingContext)
        return meanSquaredError(predicted: ŷ, expected: y)
    }
    optimizer.update(&model.allDifferentiableVariables, along: 𝛁model)
}

In [4]:
print(round(model.inferring(from: [[0, 0], [0, 1], [1, 0], [1, 1]])))

[[0.0], [1.0], [1.0], [0.0]]


In [5]:
print(model)

XOR(l1: TensorFlow.Dense<Swift.Float>(weight: [[-0.6430484, -0.006845154, 0.15502183, -0.15306444, 0.023470154, -0.06308928, 0.21356016, 0.028444273, -0.5117605, 0.900456], [0.64849, -0.06420381, 0.17531295, 0.34221375, -0.69475216, -0.6388112, 0.21544898, -0.32061544, 0.5157353, -0.89359885]], bias: [4.733124e-07, 0.0, -0.08821706, -1.6009997e-06, -0.02507543, 0.0, -0.21492144, -0.033098552, -2.4566043e-06, -1.0518217e-05], activation: (Function)), l2: TensorFlow.Dense<Swift.Float>(weight: [[0.86244786], [-0.29146093], [-0.11554975], [0.35523173], [-0.3281893], [0.6970284], [-0.26027703], [-0.25837126], [0.6338912], [1.1163903]], bias: [0.0018430026], activation: (Function)))
